In [ ]:
#Cookie Cookie2 Monty Monyt2

## The cookie problem

![title](TheCookieProblem.png)

In [1]:

from thinkbayes import Pmf
from __future__ import print_function

pmf = Pmf()
pmf.Set('Bowl 1', 0.5)
pmf.Set('Bowl 2', 0.5)
print(pmf.d)
pmf.Mult('Bowl 1', 0.75)
pmf.Mult('Bowl 2', 0.5)
print(pmf.d)
print(pmf.Prob('Bowl 1')) # P(V|1) * P(1)

pmf.Normalize()
print(pmf.d)
print(pmf.Prob('Bowl 1')) # P(V|1) * P(1) / P(V)


{'Bowl 1': 0.5, 'Bowl 2': 0.5}
{'Bowl 1': 0.375, 'Bowl 2': 0.25}
0.375
{'Bowl 1': 0.6000000000000001, 'Bowl 2': 0.4}
0.6000000000000001


In [2]:
# P(1|V) = P(V|1) * P(1) / P(V)
((.75 * .5)/
 (.75 *.5 + .5*.5))

0.6

In [30]:
from thinkbayes import Pmf


class Cookie(Pmf):
    """A map from string bowl ID to probablity."""

    def __init__(self, hypos):
        """Initialize self.

        hypos: sequence of string bowl IDs
        """
        
        self.mixes = {
        'Bowl 1':{'vanilla':0.75, 'chocolate':0.25},
        'Bowl 2':{'vanilla':0.5, 'chocolate':0.5},
        }

        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()
        

    def Likelihood(self, data, hypo):
        """The likelihood of the data under the hypothesis.

        data: string cookie type
        hypo: string bowl ID
        """
        #print(self.mixes)
        mix = self.mixes[hypo]
        #print(mix)
        like = mix[data]
        #print(like)
        return like
        
    def Update(self, data):
        """Updates the PMF with new data.

        data: string cookie type
        """
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            #print(hypo,like)
            self.Mult(hypo, like)
        self.Normalize()
        

In [32]:
hypos = ['Bowl 1', 'Bowl 2']

pmf = Cookie(hypos)

pmf.Update('vanilla')

for hypo, prob in pmf.Items():
    print(hypo, prob)


Bowl 1 0.6000000000000001
Bowl 2 0.4


In [33]:
data = 'vanilla'
hypo = 'Bowl 1'

In [34]:
mix = pmf.mixes[hypo]
like = mix[data]

In [35]:
pmf.mixes

{'Bowl 1': {'vanilla': 0.75, 'chocolate': 0.25},
 'Bowl 2': {'vanilla': 0.5, 'chocolate': 0.5}}

## The Monty Hall problem

In [40]:
from thinkbayes import Pmf


class Monty(Pmf):
    """A map from string bowl ID to probablity."""

    def __init__(self, hypos):
        """Initialize self.

        hypos: sequence of string bowl IDs
        """
        
        self.mixes = {
        'Bowl 1':{'vanilla':0.75, 'chocolate':0.25},
        'Bowl 2':{'vanilla':0.5, 'chocolate':0.5},
        }
        
        #Same as Cookies
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()
        
    def Likelihood(self, data, hypo):
        """Computes the likelihood of the data under the hypothesis.

        hypo: string name of the door where the prize is
        data: string name of the door Monty opened
        """
        if hypo == data:
            return 0
        elif hypo == 'A':
            return 0.5
        else:
            return 1
        
    def Update(self, data):
        """Updates the PMF with new data.

        data: string cookie type
        """
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            #print(hypo,like)
            self.Mult(hypo, like)
        self.Normalize()
        

In [43]:
hypos = 'ABC'
pmf = Monty(hypos)
pmf.Update('B')
pmf.Print()


A 0.3333333333333333
B 0.0
C 0.6666666666666666


In [45]:
from thinkbayes import Suite


class Monty_S(Suite):
    def Likelihood(self, data, hypo):
        """Computes the likelihood of the data under the hypothesis.

        hypo: string name of the door where the prize is
        data: string name of the door Monty opened
        """
        if hypo == data:
            return 0
        elif hypo == 'A':
            return 0.5
        else:
            return 1


In [46]:
hypos = 'ABC'
suite = Monty_S(hypos)
suite.Update('B')
suite.Print()


A 0.3333333333333333
B 0.0
C 0.6666666666666666


![title](MM.png)

In [2]:
from thinkbayes import Suite


class MM_S(Suite):
    mix94 = dict(brown=30,
                 yellow=20,
                 red=20,
                 green=10,
                 orange=10,
                 tan=10,
                 blue=0)

    mix96 = dict(blue=24,
                 green=20,
                 orange=16,
                 yellow=14,
                 red=13,
                 brown=13,
                 tan=0)

    hypoA = dict(bag1=mix94, bag2=mix96)
    hypoB = dict(bag1=mix96, bag2=mix94)

    hypotheses = dict(A=hypoA, B=hypoB)

    def Likelihood(self, data, hypo):
        """Computes the likelihood of the data under the hypothesis.

        hypo: string hypothesis (A or B)
        data: tuple of string bag, string color
        """
        bag, color = data
        mix = self.hypotheses[hypo][bag]
        like = mix[color]
        return like

In [6]:
suite = MM_S('AB')

suite.Update(('bag1', 'yellow'))
suite.Update(('bag2', 'green'))

suite.Print()


A 0.7407407407407407
B 0.2592592592592592
